In [19]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

In [20]:
# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [21]:
#Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

#Check the number of unique values in each column
charity_df[charity_cat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [22]:
#Drop the NAME column
charity_df = charity_df.drop(columns=["NAME", "EIN", "CLASSIFICATION"], axis = 1)
charity_df.head()

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1


In [23]:
#Rename APPLICATION column names
charity_df = charity_df.rename(columns ={"APPLICATION_TYPE": "application"})
charity_df.head()

,application,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1


In [24]:
#Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()

#Check the number of unique values in each column
charity_df[charity_cat].nunique()

application               17
AFFILIATION                6
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [25]:
#check the unique value counts to see if binning is required
charity_df.application.value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: application, dtype: int64

In [26]:
#print the classification count
application_counts = charity_df.application.value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: application, dtype: int64

In [27]:
# Determine which values to replace
replace_application = list(application_counts[application_counts < 500].index)

# Replace in DataFrame
for application in replace_application:
    charity_df.application = charity_df.application.replace(application,"Other")


# Check to make sure binning was successful
charity_df.application.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: application, dtype: int64

In [28]:
charity_df[charity_cat].nunique()

application               9
AFFILIATION               6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [29]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,application_Other,application_T10,application_T19,application_T3,application_T4,application_T5,application_T6,application_T7,application_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [30]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,application_Other,application_T10,application_T19,application_T3,application_T4,application_T5,application_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [31]:
# Split our preprocessed data into our features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [32]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [33]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 304       
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 355
Trainable params: 355
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Epoch 1/100
25724/25724 [==============================] - 1s 46us/sample - loss: 29076.7682 - acc: 0.5094
Epoch 2/100
25724/25724 [==============================] - 1s 49us/sample - loss: 18613.2784 - acc: 0.4972
Epoch 3/100
25724/25724 [==============================] - 1s 51us/sample - loss: 14996.0778 - acc: 0.5062
Epoch 4/100
25724/25724 [==============================] - 2s 66us/sample - loss: 10901.5541 - acc: 0.5269
Epoch 5/100
25724/25724 [==============================] - 1s 48us/sample - loss: 11886.1867 - acc: 0.5383
Epoch 6/100
25724/25724 [==============================] - 1s 44us/sample - loss: 6287.4158 - acc: 0.5213
Epoch 7/100
25724/25724 [==============================] - 1s 42us/sample - loss: 10086.5453 - acc: 0.5024
Epoch 8/100
25724/25724 [==============================] - 1s 42us/sample - loss: 9752.6738 - acc: 0.5255
Epoch 9/100
25724/25724 [==============================] - 1s 45us/sample - loss: 12768.5880 - acc: 0.5332
Epoch 10/100
25724/25724 [=============

25724/25724 [==============================] - 1s 36us/sample - loss: 0.6911 - acc: 0.5321
Epoch 79/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6911 - acc: 0.5321
Epoch 80/100
25724/25724 [==============================] - 1s 38us/sample - loss: 0.6911 - acc: 0.5321
Epoch 81/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6911 - acc: 0.5321
Epoch 82/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6911 - acc: 0.5321
Epoch 83/100
25724/25724 [==============================] - 1s 35us/sample - loss: 0.6911 - acc: 0.5321
Epoch 84/100
25724/25724 [==============================] - 1s 36us/sample - loss: 0.6911 - acc: 0.5321
Epoch 85/100
25724/25724 [==============================] - 1s 42us/sample - loss: 0.6911 - acc: 0.5321
Epoch 86/100
25724/25724 [==============================] - 1s 43us/sample - loss: 0.6911 - acc: 0.5321
Epoch 87/100
25724/25724 [==============================] - 1s 35us/sample - 

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.6910 - acc: 0.5332
Loss: 0.6909647835030848, Accuracy: 0.5331778526306152
